<a href="https://colab.research.google.com/github/tawanyb/projeto-test/blob/main/rateio_crewai_testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd

# Use pd.read_excel to read the Excel file
df = pd.read_excel("/Dados Colaboradores.xlsx")
df = pd.read_excel("/Beneficio 1 - Unimed.xlsx")

df=pd.read_excel('/Beneficio 2 - Gympass.xlsx')
df= pd.read_excel("/Ferramenta 1 - Github.xlsx")
df= pd.read_excel("/Ferramenta 2 - Google workspace.xlsx")
df

,Assinante,Documento,Data Ativacao,Valor Mensal
0,Adolfo Moreira,398.122.745-XX,2023-05-25,297.13
1,Adriana Nogueira,107.885.329-XX,2022-05-12,297.13
2,Afonso Barros,432.976.108-XX,2024-07-12,297.13
3,Alan Esteves,215.630.874-XX,2024-04-04,297.13
4,Alberto da Luz,569.041.233-XX,2023-08-25,297.13
...,...,...,...,...
57,Eduardo Portela,778.071.947-XX,2025-01-15,297.13
58,Elaine Assis,199.182.058-XX,2022-06-08,297.13
59,Elias Henriques,520.293.169-XX,2024-02-05,297.13
60,Eliane Jurema,845.601.337-XX,2024-05-14,297.13


In [ ]:
from crewai import Crew, Agent # Import Agent directly from crewai
import pandas as pd

# Agente responsável por carregar e limpar os dados das planilhas
class DataLoaderAgent:
    def __init__(self):
        self.agent = Agent(name="Data Loader", description="Carrega e prepara os dados")

    def _padroniza_cpf(self, df):
        # Remove caracteres especiais do CPF
        if "CPF" in df.columns:
            df["CPF"] = df["CPF"].astype(str).str.replace(r'\D', '', regex=True)
        return df

    def _carrega_multiplos(self, arquivos, tipo, required_cols):
        # Garante que seja uma lista de arquivos
        if not isinstance(arquivos, list):
            arquivos = [arquivos]

        dataframes = []

        for caminho in arquivos:
            try:
                df = pd.read_excel(caminho)  # Lê a planilha
                df.columns = [c.strip() for c in df.columns]  # Remove espaços extras

                # Verifica se as colunas obrigatórias existem
                if not all(c in df.columns for c in required_cols):
                    continue

                df = self._padroniza_cpf(df)

                # Filtra apenas titulares em benefícios
                if tipo == "beneficios":
                    df = df[df["Tipo"].str.upper() == "T"]

                dataframes.append(df)
            except Exception as e:
                print(f"Erro carregando {caminho}: {e}")

        # Retorna DataFrame combinado
        return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

    def run(self, inputs):
        return {
            "colaboradores": self._carrega_multiplos(inputs["colaboradores"], "colaboradores", ["CPF"]),
            "beneficios": self._carrega_multiplos(inputs["beneficios"], "beneficios", ["CPF", "Total", "Tipo"]),
            "ferramentas": self._carrega_multiplos(inputs["ferramentas"], "ferramentas", ["Valor"])
        }

In [ ]:
# Create the 'agents' directory if it doesn't exist
!mkdir -p agents

# Create empty files for the agent modules if they don't exist
!touch agents/data_loader_agent.py
!touch agents/processor_agent.py
!touch agents/report_generator_agent.py

In [ ]:
from crewai import Agent # Import Agent directly from crewai
import pandas as pd

# Agente responsável pelo processamento e rateio dos custos
class ProcessorAgent:
    def __init__(self):
        self.agent = Agent(name="Rateio Processor", description="Calcula o rateio proporcional")

    def run(self, data):
        df_colab = data["colaboradores"]
        df_benef = data["beneficios"]
        df_ferr = data["ferramentas"]

        # Soma dos benefícios por CPF
        benef_sum = df_benef.groupby("CPF")["Total"].sum().reset_index(name="Custo Benefício")

        # Junta as informações de colaboradores com os benefícios
        df = pd.merge(df_colab, benef_sum, on="CPF", how="left")

        # Calcula custo proporcional das ferramentas
        total_usuarios = len(df)
        total_ferr = df_ferr["Valor"].sum()
        valor_ferramenta_por_usuario = total_ferr / total_usuarios if total_usuarios else 0

        df["Custo Ferramenta"] = valor_ferramenta_por_usuario
        df["Custo Total"] = df["Custo Benefício"].fillna(0) + df["Custo Ferramenta"]

        return df

In [ ]:
from crewai import Agent

# Agente responsável por salvar o relatório final
class ReportGeneratorAgent:
    def __init__(self):
        self.agent = Agent(name="Relatório", description="Gera o relatório final")

    def run(self, df_resultado):
        # Salva o DataFrame em um arquivo Excel dentro da pasta outputs
        df_resultado.to_excel("outputs/relatorio_rateio.xlsx", index=False)
        print("📁 Relatório salvo em: outputs/relatorio_rateio.xlsx")


In [ ]:


# Write the DataLoaderAgent class to agents/data_loader_agent.py
%%writefile agents/data_loader_agent.py
%%writefile agents/processor_agent.py
%%writefile agents/processor_agent.py
from crewai import Agent
import pandas as pd

# Agente responsável por carregar e limpar os dados das planilhas
class DataLoaderAgent:
    def __init__(self):
        self.agent = Agent(name="Data Loader", description="Carrega e prepara os dados")

    def _padroniza_cpf(self, df):
        # Remove caracteres especiais do CPF
        if "CPF" in df.columns:
            df["CPF"] = df["CPF"].astype(str).str.replace(r'\D', '', regex=True)
        return df

    def _carrega_multiplos(self, arquivos, tipo, required_cols):
        # Garante que seja uma lista de arquivos
        if not isinstance(arquivos, list):
            arquivos = [arquivos]

        dataframes = []

        for caminho in arquivos:
            try:
                df = pd.read_excel(caminho)  # Lê a planilha
                df.columns = [c.strip() for c in df.columns]  # Remove espaços extras

                # Verifica se as colunas obrigatórias existem
                if not all(c in df.columns for c in required_cols):
                    continue

                df = self._padroniza_cpf(df)

                # Filtra apenas titulares em benefícios
                if tipo == "beneficios":
                    df = df[df["Tipo"].str.upper() == "T"]

                dataframes.append(df)
            except Exception as e:
                print(f"Erro carregando {caminho}: {e}")

        # Retorna DataFrame combinado
        return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

    def run(self, inputs):
        return {
            "colaboradores": self._carrega_multiplos(inputs["colaboradores"], "colaboradores", ["CPF"]),
            "beneficios": self._carrega_multiplos(inputs["beneficios"], "beneficios", ["CPF", "Total", "Tipo"]),
            "ferramentas": self._carrega_multiplos(inputs["ferramentas"], "ferramentas", ["Valor"])
        }

# Write the ProcessorAgent class to agents/processor_agent.py

from crewai.agents import Agent
import pandas as pd

# Agente responsável pelo processamento e rateio dos custos
class ProcessorAgent:
    def __init__(self):
        self.agent = Agent(name="Rateio Processor", description="Calcula o rateio proporcional")

    def run(self, data):
        df_colab = data["colaboradores"]
        df_benef = data["beneficios"]
        df_ferr = data["ferramentas"]

        # Soma dos benefícios por CPF
        benef_sum = df_benef.groupby("CPF")["Total"].sum().reset_index(name="Custo Benefício")

        # Junta as informações de colaboradores com os benefícios
        df = pd.merge(df_colab, benef_sum, on="CPF", how="left")

        # Calcula custo proporcional das ferramentas
        total_usuarios = len(df)
        total_ferr = df_ferr["Valor"].sum()
        valor_ferramenta_por_usuario = total_ferr / total_usuarios if total_usuarios else 0

        df["Custo Ferramenta"] = valor_ferramenta_por_usuario
        df["Custo Total"] = df["Custo Benefício"].fillna(0) + df["Custo Ferramenta"]

        return df

# Write the ReportGeneratorAgent class to agents/report_generator_agent.py


from crewai import Agent

# Agente responsável por salvar o relatório final
class ReportGeneratorAgent:
    def __init__(self):
        self.agent = Agent(name="Relatório", description="Gera o relatório final")

    def run(self, df_resultado):
        # Salva o DataFrame em um arquivo Excel dentro da pasta outputs
        !mkdir -p outputs # Ensure outputs directory exists
        df_resultado.to_excel("outputs/relatorio_rateio.xlsx", index=False)
        print("📁 Relatório salvo em: outputs/relatorio_rateio.xlsx")

Overwriting agents/data_loader_agent.py


In [ ]:
# agents/data_loader_agent.py
import pandas as pd

class DataLoaderAgent:
    def run(self, inputs):
        data = {}

        # Carrega colaboradores
        data['colaboradores'] = pd.read_excel(inputs['colaboradores'])

        # Carrega benefícios (lista de arquivos)
        beneficios = []
        for file in inputs['beneficios']:
            df = pd.read_excel(file)
            beneficios.append(df)
        data['beneficios'] = beneficios

        # Carrega ferramentas (lista de arquivos)
        ferramentas = []
        for file in inputs['ferramentas']:
            df = pd.read_excel(file)
            ferramentas.append(df)
        data['ferramentas'] = ferramentas

        return data


In [ ]:
# agents/processor_agent.py
import pandas as pd

class ProcessorAgent:
    def run(self, data):
        colaboradores = data['colaboradores']

        # Concatenar todos os benefícios em um DataFrame só
        beneficios_df = pd.concat(data['beneficios'], ignore_index=True)

        # Concatenar todas as ferramentas em um DataFrame só
        ferramentas_df = pd.concat(data['ferramentas'], ignore_index=True)

        # Exemplo: calcular total de benefícios e ferramentas por colaborador
        # Supondo que ambos tenham uma coluna 'Colaborador' e 'Valor' ou equivalente

        beneficios_agg = beneficios_df.groupby('Colaborador')['Valor'].sum().reset_index()
        beneficios_agg.rename(columns={'Valor': 'Total_Beneficios'}, inplace=True)

        ferramentas_agg = ferramentas_df.groupby('Colaborador')['Valor'].sum().reset_index()
        ferramentas_agg.rename(columns={'Valor': 'Total_Ferramentas'}, inplace=True)

        # Merge tudo com colaboradores
        resultado = colaboradores.merge(beneficios_agg, on='Colaborador', how='left')
        resultado = resultado.merge(ferramentas_agg, on='Colaborador', how='left')

        # Substituir NaN por zero para os totais
        resultado['Total_Beneficios'] = resultado['Total_Beneficios'].fillna(0)
        resultado['Total_Ferramentas'] = resultado['Total_Ferramentas'].fillna(0)

        return resultado


In [ ]:
import pandas as pd

class DataLoaderAgent:
    def run(self, inputs):
        data = {}

        # Carregar colaboradores
        data['colaboradores'] = pd.read_excel(inputs['colaboradores'])

        # Carregar benefícios
        beneficios = []
        for arquivo in inputs['beneficios']:
            df = pd.read_excel(arquivo)
            beneficios.append(df)
        data['beneficios'] = beneficios

        # Carregar ferramentas
        ferramentas = []
        for arquivo in inputs['ferramentas']:
            df = pd.read_excel(arquivo)
            ferramentas.append(df)
        data['ferramentas'] = ferramentas

        return data


In [ ]:
import pandas as pd

class ProcessorAgent:
    def run(self, data):
        colaboradores = data['colaboradores']

        # Concatenar todas planilhas de benefícios e ferramentas
        beneficios_df = pd.concat(data['beneficios'], ignore_index=True)
        ferramentas_df = pd.concat(data['ferramentas'], ignore_index=True)

        # Somar valores de benefícios por colaborador
        beneficios_agg = beneficios_df.groupby('Colaborador')['Valor'].sum().reset_index()
        beneficios_agg.rename(columns={'Valor': 'Total_Beneficios'}, inplace=True)

        # Somar valores de ferramentas por colaborador
        ferramentas_agg = ferramentas_df.groupby('Colaborador')['Valor'].sum().reset_index()
        ferramentas_agg.rename(columns={'Valor': 'Total_Ferramentas'}, inplace=True)

        # Mesclar tudo com dados dos colaboradores
        df_final = colaboradores.merge(beneficios_agg, on='Colaborador', how='left')
        df_final = df_final.merge(ferramentas_agg, on='Colaborador', how='left')

        # Substituir NaNs por zero
        df_final['Total_Beneficios'] = df_final['Total_Beneficios'].fillna(0)
        df_final['Total_Ferramentas'] = df_final['Total_Ferramentas'].fillna(0)

        # Exemplo de rateio: custo total por colaborador = soma dos benefícios + ferramentas
        df_final['Custo_Total'] = df_final['Total_Beneficios'] + df_final['Total_Ferramentas']

        return df_final


In [ ]:
import os

class ReportGeneratorAgent:
    def run(self, df):
        os.makedirs('output', exist_ok=True)
        output_path = 'output/relatorio_rateio.xlsx'
        df.to_excel(output_path, index=False)
        print(f"Relatório salvo em {output_path}")


In [ ]:
# Write the DataLoaderAgent class to agents/data_loader_agent.py
%%writefile agents/data_loader_agent.py
%%writefile agents/processor_agent.py
%%writefile agents/processor_agent.py
from crewai import Agent
import pandas as pd

# Agente responsável por carregar e limpar os dados das planilhas
class DataLoaderAgent:
    # ... (content of DataLoaderAgent)
# ... (content of ProcessorAgent)

Overwriting agents/data_loader_agent.py


In [ ]:
# Write the DataLoaderAgent class to agents/data_loader_agent.py
%%writefile agents/data_loader_agent.py
import pandas as pd
from crewai.agents import Agent

# Agente responsável por carregar e limpar os dados das planilhas
class DataLoaderAgent:
    def __init__(self):
        # Agent instantiation is not needed in the class intended for import
        # unless it's part of the agent's run method logic or a shared resource.
        # For this specific usage pattern (instantiating the class and then using agent.run),
        # the agent itself is not defined directly within this class file.
        # However, based on the original code structure where the agent is instantiated in the
        # CostAllocationCrew, we'll keep the focus on the data loading logic in this file.
        pass # No Agent init needed here

    def _padroniza_cpf(self, df):
        # Remove caracteres especiais do CPF
        if "CPF" in df.columns:
            df["CPF"] = df["CPF"].astype(str).str.replace(r'\D', '', regex=True)
        return df

    def _carrega_multiplos(self, arquivos, tipo, required_cols):
        # Garante que seja uma lista de arquivos
        if not isinstance(arquivos, list):
            arquivos = [arquivos]

        dataframes = []

        for caminho in arquivos:
            try:
                df = pd.read_excel(caminho)  # Lê a planilha
                df.columns = [c.strip() for c in df.columns]  # Remove espaços extras

                # Verifica se as colunas obrigatórias existem
                if not all(c in df.columns for c in required_cols):
                    print(f"Skipping {caminho} due to missing required columns: {required_cols}")
                    continue

                df = self._padroniza_cpf(df)

                # Filtra apenas titulares em benefícios
                if tipo == "beneficios":
                     # Ensure 'Tipo' column exists and is string type before filtering
                    if "Tipo" in df.columns:
                        df = df[df["Tipo"].astype(str).str.upper() == "T"]
                    else:
                        print(f"Warning: 'Tipo' column not found in {caminho}. Skipping filtering.")


                dataframes.append(df)
            except FileNotFoundError:
                 print(f"Erro: Arquivo não encontrado em {caminho}")
            except Exception as e:
                print(f"Erro carregando {caminho}: {e}")

        # Retorna DataFrame combinado
        return pd.concat(dataframes, ignore_index=True) if dataframes else pd.DataFrame()

    def run(self, inputs):
        # Ensure inputs are lists even if only one file is provided
        colab_input = inputs.get("colaboradores")
        benef_input = inputs.get("beneficios", [])
        ferr_input = inputs.get("ferramentas", [])

        if isinstance(colab_input, str):
             colab_input = [colab_input]
        elif not isinstance(colab_input, list):
             colab_input = [] # Handle case where collaborators input is missing or not a list/string


        return {
            "colaboradores": self._carrega_multiplos(colab_input, "colaboradores", ["CPF"]),
            "beneficios": self._carrega_multiplos(benef_input, "beneficios", ["CPF", "Total", "Tipo"]),
            "ferramentas": self._carrega_multiplos(ferr_input, "ferramentas", ["Valor"])
        }

# Write the ProcessorAgent class to agents/processor_agent.py
%%writefile agents/processor_agent.py
import pandas as pd
from crewai.agents import Agent

# Agente responsável pelo processamento e rateio dos custos
class ProcessorAgent:
    def __init__(self):
        # Agent instantiation not needed here based on CostAllocationCrew structure
        pass

    def run(self, data):
        df_colab = data.get("colaboradores", pd.DataFrame()) # Provide default empty DataFrame
        df_benef = data.get("beneficios", pd.DataFrame())
        df_ferr = data.get("ferramentas", pd.DataFrame())

        # Check if required dataframes are empty
        if df_colab.empty:
            print("Colaboradores data is empty. Cannot process.")
            return pd.DataFrame()

        # Ensure 'CPF' exists in beneficiarios before grouping
        if "CPF" in df_benef.columns and "Total" in df_benef.columns:
             # Soma dos benefícios por CPF
            benef_sum = df_benef.groupby("CPF")["Total"].sum().reset_index(name="Custo Benefício")
        else:
             print("Beneficios data missing 'CPF' or 'Total' columns. Cannot calculate benefits sum.")
             benef_sum = pd.DataFrame({'CPF':[], 'Custo Benefício':[]}) # Create an empty DataFrame


        # Junta as informações de colaboradores com os benefícios
        df = pd.merge(df_colab, benef_sum, on="CPF", how="left")

        # Ensure 'Valor' exists in ferramentas before summing
        if "Valor" in df_ferr.columns:
            # Calcula custo proporcional das ferramentas
            total_usuarios = len(df)
            total_ferr = df_ferr["Valor"].sum()
            valor_ferramenta_por_usuario = total_ferr / total_usuarios if total_usuarios else 0
        else:
             print("Ferramentas data missing 'Valor' column. Cannot calculate tool cost.")
             valor_ferramenta_por_usuario = 0

        df["Custo Ferramenta"] = valor_ferramenta_por_usuario
        df["Custo Benefício"] = df["Custo Benefício"].fillna(0) # Ensure Custo Benefício is filled even if merge didn't add all CPFs
        df["Custo Total"] = df["Custo Benefício"] + df["Custo Ferramenta"]


        return df

# Write the ReportGeneratorAgent class to agents/report_generator_agent.py
%%writefile agents/report_generator_agent.py
import os
from crewai.agents import Agent
import pandas as pd

# Agente responsável por salvar o relatório final
class ReportGeneratorAgent:
    def __init__(self):
        # Agent instantiation not needed here based on CostAllocationCrew structure
        pass

    def run(self, df_resultado):
        if df_resultado.empty:
            print("Resultado DataFrame is empty. Cannot generate report.")
            return

        # Ensure outputs directory exists
        output_dir = "outputs"
        os.makedirs(output_dir, exist_ok=True)

        output_path = os.path.join(output_dir, "relatorio_rateio.xlsx")
        df_resultado.to_excel(output_path, index=False)
        print(f"📁 Relatório salvo em: {output_path}")

Overwriting agents/data_loader_agent.py
